In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os
go_term_gene_matrix = pd.read_csv(".../Atlas/MN/go_sets_human.csv", index_col="Unnamed: 0")
seed=10
os.chdir(".../Atlas/objects")
sc.logging.print_versions()
!pip list 

In [ ]:
adata = sc.read("Atlas6.6_3000HVG_integrated_V2.h5ad")

In [ ]:
# Filter genes in AnnData object and GO-term_gene table
genes_to_keep = adata.var_names.intersection(go_term_gene_matrix.index)
adata_filtered = adata[:, genes_to_keep]
go_term_gene_matrix = go_term_gene_matrix.loc[genes_to_keep]



# Subsetting gene sets with more than 10 genes
go_term_gene_matrix = go_term_gene_matrix.loc[:, (go_term_gene_matrix == 1).sum() > 10]


mean_expression_df = pd.DataFrame(index=adata.obs_names)


mean_expression_dict = {}

for go_term in go_term_gene_matrix.columns:

    # Define an example gene set (use the first 10 genes in the dataset)
    gene_set = go_term_gene_matrix.index[go_term_gene_matrix[go_term] == 1].tolist()

    # Get the expression values for all cells and the gene set
    cell_expr = adata_filtered.raw[:, gene_set].X

    mean_expression = np.asarray(cell_expr.mean(axis=1)).flatten()
    
    # Add the mean_expression values to the dictionary with the key as go_term
    mean_expression_dict[go_term] = mean_expression

# Convert the dictionary to a DataFrame
mean_expression_df = pd.DataFrame(mean_expression_dict, index=adata.obs_names)

In [ ]:
import anndata
from scipy.sparse import csr_matrix

# Convert the mean_expression_df DataFrame to a sparse matrix
mean_expression_sparse = csr_matrix(mean_expression_df.values)



In [ ]:
# Create a new AnnData object with the mean expression matrix as adata_new.X and copy UMAP information
adata_new = anndata.AnnData(X=mean_expression_sparse, obs=adata.obs, var=pd.DataFrame(index=mean_expression_df.columns), obsm=adata.obsm)


In [ ]:
adata_new.write(".../Atlas/objects/Atlas6.6_Genesets.h5ad")